In [12]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib

  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-aa3y4mnm
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-aa3y4mnm
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit b5e74674973de4a97431561d7723d9b29629a695
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
from pathlib import Path 
import gc    
import torch
import os
import torch  
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)   

In [14]:
%cd /root/text-to-mesh

/root/text-to-mesh


In [15]:
autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,    
        codebook_size = 2048, 
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8,
    
    attn_decoder_depth  = 4,
    attn_encoder_depth = 2
    ).to("cuda")     

In [16]:
dataset = MeshDataset.load("./objverse_250f_490.7M_all_17561_labels_568425_5_min_x5_aug.npz")  
dataset2 = MeshDataset.load("./objverse_250f_98.1M_all_17561_labels_113685_5_min_x1_aug.npz")
dataset.data.extend(dataset2.data)  
dataset2 = MeshDataset.load("./shapenet_250f_2.2M_84_labels_2156_10_min_x1_aug.npz")  
dataset.data.extend(dataset2.data)  
dataset2 = MeshDataset.load("./shapenet_250f_21.9M_84_labels_21560_10_min_x10_aug.npz")  
dataset.data.extend(dataset2.data) 
dataset.sort_dataset_keys()
print("length", len(dataset.data))

def format_value(value):
    if value >= 1_000_000_000:
        return f"{value / 1_000_000_000:.1f}B"
    elif value >= 1_000_000:
        return f"{value / 1_000_000:.1f}M"
    else:
        return f"{value}"

tokens = 0
for item in dataset.data:
    tokens += len(item['faces']) * 6 
total_tokens = format_value(tokens)
print("Tokens:", total_tokens)

[MeshDataset] Loaded 568425 entrys
[MeshDataset] Created from 568425 entrys
[MeshDataset] Loaded 113685 entrys
[MeshDataset] Created from 113685 entrys
[MeshDataset] Loaded 2156 entrys
[MeshDataset] Created from 2156 entrys
[MeshDataset] Loaded 21560 entrys
[MeshDataset] Created from 21560 entrys
length 705826
Tokens: 612.9M


In [17]:
pkg = torch.load("./2k_autoencoder_490M__recon_0.3665_commit_-0.7556.pt") 
autoencoder.load_state_dict(pkg['model'])
for param in autoencoder.parameters():
    param.requires_grad = True

In [19]:
batch_size=32 # The batch size should be max 64.
grad_accum_every =2
# So set the maximal batch size (max 64) that your VRAM can handle and then use grad_accum_every to create a effective batch size of 64, e.g  16 * 4 = 64
learning_rate = 1e-3 # Start with 1e-3 then at stagnation around 0.35, you can lower it to 1e-4.

autoencoder.commit_loss_weight = 0.5 # Set dependant on the dataset size, on smaller datasets, 0.1 is fine, otherwise try from 0.25 to 0.4.a 
 
loss = autoencoder_trainer.train(480, diplay_graph= True)   

Epoch 1/480: 100%|██████████| 22057/22057 [50:51<00:00,  7.23it/s, commit_loss=-0.864, loss=-0.0782, recon_loss=0.354]  


Epoch 1 average loss: 0.01600448033540729 recon loss: 0.3757: commit_loss -0.7194


Epoch 2/480: 100%|██████████| 22057/22057 [50:49<00:00,  7.23it/s, commit_loss=-0.795, loss=-0.0384, recon_loss=0.359]  


Epoch 2 average loss: -0.004100833749378744 recon loss: 0.3715: commit_loss -0.7513


Epoch 3/480: 100%|██████████| 22057/22057 [50:43<00:00,  7.25it/s, commit_loss=-0.86, loss=-0.0801, recon_loss=0.35]    


Epoch 3 average loss: -0.013985901990605702 recon loss: 0.3693: commit_loss -0.7665


Epoch 4/480:  87%|████████▋ | 19293/22057 [43:53<06:30,  7.07it/s, commit_loss=-0.677, loss=0.0237, recon_loss=0.362]   

In [ ]:
pkg = dict( model = autoencoder.state_dict(), ) 
import datetime
# now_utc = datetime.datetime.now(datetime.timezone.utc).isoformat().replace(":", "_").replace("+", "_") 
#torch.save(pkg, f"./16k_autoencoder_229M_0.338.pt")
torch.save(pkg, f"./2k_autoencoder.pt")